**Dependencies**

In [ ]:
# Installing python dependencies
%pip install openai jsonlines

In [2]:
# Import Libraries

import json
from openai import OpenAI

**Module**

In [7]:
%pip install nbformat

     -------------------------------------- 77.6/77.6 kB 215.2 kB/s eta 0:00:00
     -------------------------------------- 84.7/84.7 kB 339.8 kB/s eta 0:00:00
     -------------------------------------- 188.7/188.7 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Authentication**

In [ ]:
# Initializing GPT Api
api_key = ""
client = OpenAI(api_key=api_key)

**Dataset**

In [ ]:
#Trip dataset
%run dataset.ipynb

print(data)

In [ ]:
#Converting the database to a suitable format\
import jsonlines

jsonl_filename = "output.jsonl"

#save as JSONL (Json line file)
with jsonlines.open(jsonl_filename,"w") as writer:
    writer.write_all(data)

In [ ]:
# Insert the training data into context
file = client.files.create(
  file = open(jsonl_filename, "rb"),
  purpose = "fine-tune"
)


# Extracting the dataset id
dataset = file.id
print(dataset)

**Training our model**

In [ ]:
# Starting a fine-tuning job
trained_model = client.fine_tuning.jobs.create(
  training_file=dataset, 
  model="gpt-3.5-turbo",
hyperparameters={
    "n_epochs":1
  }
)

# Retrieving The model Training Status
jobstatus = client.fine_tuning.jobs.retrieve(trained_model.id)
print(jobstatus)

In [ ]:
# Retrieving The model Training Status
jobstatus = client.fine_tuning.jobs.retrieve(trained_model.id)
print(jobstatus)

**Extracting Model**

In [ ]:
# Retrieving The fine-tuned model id
fine_tuned_model = jobstatus.fine_tuned_model
print(fine_tuned_model)

**Testing**

In [ ]:
client.chat.completions.create(
model = fine_tuned_model,
    messages = [
        {"role": "System", "content":  }
    ]
)

**Keep Track of Conversation History**

In [ ]:
#Keeping track of chat history
# Keeping track of chat history
history = []
def rude_guy(message):
    # adding to history
    history.append({"role": "system", "content": "Marv is a rude chatbot that is also blunt."})
    history.append({"role": "user", "content": message})

    # Processing the history and new message
    response = client.chat.completions.create(
      model=fine_tuned_model,
      messages=history
    )

    # getting the response
    response_msg = response.choices[0].message.content

    # adding the response to the history
    history.append({"role": "system", "content": response_msg})

    # returning the response to the user
    return response_msg

In [ ]:
print(rude_guy("What does hakuna mata means? it means the world is crazy"))